In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.externals.six import StringIO
import pydotplus

In [2]:
reviews_csv = "Hotel_Reviews.csv"
csv_file = pd.read_csv(reviews_csv)
csv_file.head(5)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [3]:
csv_file = csv_file.drop(['Hotel_Address','Additional_Number_of_Scoring','Review_Date','Hotel_Name','lat','lng'], axis=1)
csv_file = csv_file.drop(['Reviewer_Nationality','Negative_Review','Positive_Review','Tags','days_since_review'], axis=1)

In [4]:
csv_file.insert(6,'Recommand',1)
csv_file.head(5)

,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Recommand
0,7.7,397,1403,11,7,2.9,1
1,7.7,0,1403,105,7,7.5,1
2,7.7,42,1403,21,9,7.1,1
3,7.7,210,1403,26,1,3.8,1
4,7.7,140,1403,8,3,6.7,1


In [5]:
constraint_1 = csv_file['Review_Total_Negative_Word_Counts']>300
constraint_2 = csv_file['Review_Total_Positive_Word_Counts']<20
constraint_3 = csv_file['Review_Total_Negative_Word_Counts']+csv_file['Review_Total_Positive_Word_Counts']<50
constraint_4 = csv_file['Review_Total_Negative_Word_Counts']>2*csv_file['Review_Total_Positive_Word_Counts']
constraint_5 = csv_file['Total_Number_of_Reviews_Reviewer_Has_Given']<2
constraint_6 = csv_file['Average_Score']<3
constraint_7 = csv_file['Reviewer_Score']>9.5
constraint_8 = csv_file['Reviewer_Score']<1
constraint_9 = abs(csv_file['Average_Score']-csv_file['Reviewer_Score'])>7
constraint_10 = csv_file['Total_Number_of_Reviews']<300

In [6]:
csv_file.loc[constraint_1,'Recommand'] = 0
csv_file.loc[constraint_2,'Recommand'] = 0
csv_file.loc[constraint_3,'Recommand'] = 0
csv_file.loc[constraint_4,'Recommand'] = 0
csv_file.loc[constraint_5,'Recommand'] = 0
csv_file.loc[constraint_6,'Recommand'] = 0
csv_file.loc[constraint_7,'Recommand'] = 0
csv_file.loc[constraint_8,'Recommand'] = 0
csv_file.loc[constraint_9,'Recommand'] = 0
csv_file.loc[constraint_10,'Recommand'] = 0
csv_file.head(5)

,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Recommand
0,7.7,397,1403,11,7,2.9,0
1,7.7,0,1403,105,7,7.5,1
2,7.7,42,1403,21,9,7.1,1
3,7.7,210,1403,26,1,3.8,0
4,7.7,140,1403,8,3,6.7,0


In [7]:
X = csv_file.drop('Recommand', axis=1)
Y = csv_file['Recommand']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [8]:
dtree=DecisionTreeClassifier(max_depth=5, min_samples_split=20)
dtree.fit(X_train, Y_train)

dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data, filled=True, feature_names=list(X_train), class_names=['Recommand','Not Recommand'], special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")

True

In [9]:
Y_predict = dtree.predict(X_test)
Y_predict

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [10]:
accuracy_score(Y_test, Y_predict)

0.9857195486097646